In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/data.zip

Streaming output truncated to the last 5000 lines.
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function56/diff_OpenSSL_1_1_0l_vs_OpenSSL_1_1_1-pre1@binary_new.txt  
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function56/diff_OpenSSL_1_1_0l_vs_OpenSSL_1_1_1-pre1@binary_old.txt  
   creating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function57/
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function57/diff_OpenSSL_1_1_0l_vs_OpenSSL_1_1_1-pre1.txt  
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function57/diff_OpenSSL_1_1_0l_vs_OpenSSL_1_1_1-pre1@binary_new.txt  
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function57/diff_OpenSSL_1_1_0l_vs_OpenSSL_1_1_1-pre1@binary_old.txt  
   creating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function58/
  inflating: oldestVersion_vs_newestVersion_code/openssl/ssl/ssl_lib.c/function58/diff_Op

In [ ]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00


In [ ]:
cd /content/drive/MyDrive/Colab/Research/

/content/drive/MyDrive/Colab/Research


In [ ]:
from transformers import AutoTokenizer, AutoModel, RobertaConfig, set_seed
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
import logging
import copy
from collections import defaultdict

import numpy as np
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")
pretrain_model = AutoModel.from_pretrained("neulab/codebert-c")
config = RobertaConfig.from_pretrained("neulab/codebert-c")

vocab = tokenizer.get_vocab()
device = torch.device('cuda')
dataset_path = "/content/oldestVersion_vs_newestVersion_code"
model_path = "/content/drive/MyDrive/Colab/Research/model_state"
evaluation_path = "/content/drive/MyDrive/Colab/Research/evaluation"
seed = 628
set_seed(seed)
logger = logging.getLogger(__name__)
logging.basicConfig(level = logging.INFO,format='%(levelname)-5s [%(filename)s:%(lineno)d] %(message)s')

Some weights of the model checkpoint at neulab/codebert-c were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-c and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def getAllPath():
    path_list = []
    for dirs in tqdm(os.walk(dataset_path)):
        if len(dirs[-1]) > 0:
          for f in dirs[-1]:
            if f.startswith('diff'):
              path_list.append(f'{dirs[0]}{os.sep}{f}')
    return path_list

def yield_functions(shuffle=False, seed=None, split_ratio=None, mode=None, finetune=False):
    path_list = getAllPath()
    r = random.Random(seed)

    if split_ratio is not None:
        split_point = int(len(path_list) * split_ratio)
        if mode == 'train':
            path_list = path_list[:split_point]
        elif mode == 'test':
            path_list = path_list[split_point:]

    if shuffle:
        r.shuffle(path_list)

    maxSequenceLength = 512
    for fp in tqdm(path_list):
        token_list = {}

        lno = 1
        func_length = 0
        func_name = None
        identifier_list = []
        func_name_new = None
        func_name_old = None
        libName = 'None'
        try:
            with open(fp,"r") as file:
                codeLines = file.readlines()
                if 'openssl' == fp.split(os.sep)[2]:
                  libName = 'openssl'
                else:
                  libName = 'FFmpeg'

                for i in range(len(codeLines)):
                    # empty line
                    line = codeLines[i]
                    if i == 0 and line.lstrip().rstrip() == '':
                      print('Yes')
                    if i == 0:
                      line = line.lstrip().rstrip().split("#@#")
                      func_name_new = line[0].lstrip().rstrip()
                      func_name_old = line[1].lstrip().rstrip()
                      continue
                    if '@@##$$##@@' in line:
                      line = line.split("@@##$$##@@")[-1]
                    if len(line.lstrip().strip().rstrip()) == 0:
                        continue

                    #line = line.lstrip().rstrip()
                    line = line.rstrip()
                    if line.startswith("+") or line.startswith("-"):
                        # remove space before the code starts
                        #line = line[0] + " " + line[1:].lstrip()
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        #print(line)
                        func_length += len(token_list[lno])
                    else:
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        func_length += len(token_list[lno])
                        #print(line)
                    lno += 1
            file.close()
            if func_name_new == 'None' and func_name_old =='None':
              continue
            if func_length > 511:
              continue
            """
            codeT5:
                <s>:1
                <pad>:0
                +:15
                -:17
            """

            new = [0]
            old = [0]
            d = [12,2744] # represent - and +
            #new = [0] + [token for line in sorted(token_list.keys()) if token_list[line][0] == 2744 or token_list[line][0] not in d for token in token_list[line]]
            #old = [0] + [token for line in sorted(token_list.keys()) if token_list[line][0] == 12 or token_list[line][0] not in d for token in token_list[line]]
            for line in sorted(token_list.keys()):
              if token_list[line][0] == 2744:
                new.extend(token_list[line][1:])
              elif token_list[line][0] == 12:
                old.extend(token_list[line][1:])
              else:
                new.extend(token_list[line])
                old.extend(token_list[line])

            diff = [0] + [token for line in sorted(token_list.keys()) for token in token_list[line]]
            l = {'diff':diff,'new':new,'old':old}

            for version, func_tokens in l.items():
                if finetune:
                    yield fp, func_name_new, func_name_old, func_tokens, version
                else:
                    yield fp, func_name_new, func_name_old, func_tokens
        except Exception as e:
            print(e)
            print(fp)

def finetune_data_loader_codeBert(d, func_len, batch_size, mode):
    pad_id = d['<pad>']
    output_d = defaultdict(lambda: defaultdict(list))
    total_functions = 0
    for path, func_name_new, func_name_old, token_id_list, version in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,finetune=True):
        k = path.split(os.sep)
        output_d[k[2]][path].append(token_id_list)
        # if len(output_d[k[2]][path]) > 3:
        #   print()
        #   print(len(output_d[k[2]][path]))
        #   print(func_name)
        #   print(path)
        #   input()
        total_functions += 1
        #print(f'{k[-4]}: add function to {func_name} Length:{len(output_d[k[-4]][func_name])}')
    print(total_functions)
    padding_list = []
    output = []
    processed_functions = 0
    for func_dict in output_d.values():
        for func_name, func_list in tqdm(func_dict.items()):
            for token_id_list in func_list:
                copy_token_id_list = copy.deepcopy(token_id_list)
                padding_list.append([1] * len(copy_token_id_list))
                if (l := len(copy_token_id_list)) < func_len:
                    copy_token_id_list += [pad_id] * (func_len - l)
                    padding_list[-1] += [0] * (func_len - l)
                output.append(copy_token_id_list)
            if len(output) >= batch_size or len(output) + processed_functions == total_functions:
                processed_functions += len(output)
                data = np.asarray(output, dtype=np.int64)
                attention_mask = np.asarray(padding_list, dtype=np.int64)
                output.clear()
                padding_list.clear()
                yield data, attention_mask

def test_data_loader_codeBert_recall(d, func_len, batch_size, mode, poolsize):
    pad_id = d['<pad>']
    all_functions = {}
    positive_pairs = {}
    diff_functions = []
    total_functions = 0

    for path, func_name_new, func_name_old, token_id_list, version in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode, finetune=True):
        k = path.split(os.path.sep)
        func_name = f'{k[2]}-{k[3]}-{k[4]}-{k[5]}-{k}-{version}'
        all_functions[func_name] = token_id_list
        if version == 'diff' or version == 'new':
            positive_pairs[func_name] = token_id_list
        else:
            positive_pairs[func_name] = token_id_list
            diff_functions.append(positive_pairs)
            positive_pairs = {}
        total_functions += 1
        #print(f'{k[-4]}: add function to {func_name} Length:{len(output_d[k[-4]][func_name])}')
    padding_list = []
    output = []
    labels = []

    processed_functions = 0
    #count = 0
    for positive_pair in diff_functions:
        pk = list(positive_pair.keys())
        function_names = list(all_functions.keys())
        start = True
        while start:
            func_list = [positive_pair[pk[0]],positive_pair[pk[1]]]
            while len(func_list) < batch_size:
                fname = random.choice(function_names)
                if fname not in positive_pair:
                    func_list.append(all_functions[fname])
                    function_names.remove(fname)

            for token_id_list in func_list:
                padding_list.append([True] * len(token_id_list))
                if (l := len(token_id_list)) < func_len:
                    token_id_list += [pad_id] * (func_len - l)
                    padding_list[-1] += [False] * (func_len - l)
                output.append(token_id_list)

            processed_functions += len(output) - 2
            data = np.asarray(output, dtype=np.int64)
            attention_mask = np.asarray(padding_list, dtype=bool)
            padding_list.clear()
            output.clear()
            if processed_functions % poolsize == 0:
                start = False
            yield data, attention_mask, labels, pk[0]

In [ ]:
def getAllPath():
    path_list = []
    for dirs in tqdm(os.walk(dataset_path)):
        if len(dirs[-1]) > 0:
            d = {}
            for f in dirs[-1]:
                if not 'binary_old' in f and not 'binary_new' in f:
                  d[f'{dirs[0]}{os.sep}{f}'] = []

            for f in dirs[-1]:
              if 'binary_old' in f or 'binary_new' in f:
                for k in d:
                  if f.split("@")[0] + ".txt" == k.split(os.sep)[-1]:
                    d[k].append(f'{dirs[0]}{os.sep}{f}')
            for k in d:
              filelist = []
              filelist.append(k)
              filelist.extend(d[k])

              if len(filelist) == 3:
                path_list.append(filelist)
                fl = copy.deepcopy(filelist)
                fl = [i.split(os.sep)[-1] for i in fl]


    print(len(path_list))
    return path_list

def psu_tokenize(p):
    token_list = {}
    lno = 1
    func_length = 0
    with open(p,"r") as file:
        for line in file:
            # empty line
            if len(line.lstrip().strip().rstrip()) == 0:
                continue

            line = line.lstrip().rstrip()
            token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
            token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
            func_length += len(token_list[lno])
            lno += 1
    file.close()

    if func_length > 511:
        return 'None'
    res = [0] + [token for line in sorted(token_list.keys()) for token in token_list[line]]

    return res

def yield_functions(shuffle=False, seed=None, split_ratio=None, mode=None, testing=None):
    path_list = getAllPath()
    total = len(path_list)
    r = random.Random(seed)

    if split_ratio is not None:
        split_point = int(len(path_list) * split_ratio)
        if mode == 'train':
            path_list = path_list[:split_point]
        elif mode == 'test':
            path_list = path_list[split_point:]

    if shuffle:
      r.shuffle(path_list)
    maxSequenceLength = 512
    count = 0
    for file_list in tqdm(path_list):
        diff_fp = ''
        binary_old_fp = ''
        binary_new_fp = ''
        for fp in file_list:
            if 'binary_old' not in fp and 'binary_new' not in fp:
                diff_fp = fp
            elif 'binary_old' in fp:
                binary_old_fp = fp
            elif 'binary_new' in fp:
                binary_new_fp = fp

        token_list = {}
        lno = 1
        func_length = 0
        func_name_new = None
        func_name_old = None
        try:
            with open(diff_fp,"r") as file:
                codeLines = file.readlines()
                for i in range(len(codeLines)):
                    # empty line
                    line = codeLines[i]
                    if i == 0:
                      line = line.lstrip().rstrip().split("#@#")
                      func_name_new = line[0].lstrip().rstrip()
                      func_name_old = line[1].lstrip().rstrip()
                      continue
                    if '@@##$$##@@' in line:
                      line = line.split("@@##$$##@@")[-1]
                    if len(line.lstrip().strip().rstrip()) == 0:
                        continue

                    #line = line.lstrip().rstrip()
                    line = line.rstrip()

                    if line.startswith("+") or line.startswith("-"):
                        # remove space before the code starts
                        #line = line[0] + " " + line[1:].lstrip()
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        func_length += len(token_list[lno])
                    else:
                        token_list[lno] = tokenizer.tokenize(line) + [tokenizer.sep_token]
                        token_list[lno] = tokenizer.convert_tokens_to_ids(token_list[lno])
                        func_length += len(token_list[lno])
                    lno += 1
            file.close()

            if func_length > 511:
                count += 1
                continue

            d = [12,2744] # represent - and +
            diff = [0] + [token for line in sorted(token_list.keys()) for token in token_list[line]]

            binary_new = 'None'
            binary_old = 'None'
            if binary_new_fp != '':
                binary_new = psu_tokenize(binary_new_fp)
            if binary_old_fp != '':
                binary_old = psu_tokenize(binary_old_fp)

            if binary_new == 'None' or binary_old == 'None':
              continue

            if binary_new == binary_old:
              continue
            if not testing:
              yield diff, binary_new, binary_old, diff_fp
            else:
              ls = {'diff':diff,'new':binary_new,'old':binary_old}
              for version,token_id_list in ls.items():
                yield diff_fp, token_id_list, version

        except Exception as e:
            print(e)
            print(fp)

def finetune_data_loader_codeBert(tokenizer,batch_size, func_len, mode):
    s_pad_id = tokenizer.pad_token_id

    output_d = {}
    allFunctions = {}
    total_functions = 0
    r = random.Random(seed)
    count_new = 0
    count_old = 0
    for diff, binary_new, binary_old, diff_fp in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode):
        output_d[diff_fp] = [diff, binary_new, binary_old]
        if binary_new != 'None':
            allFunctions[f'{diff_fp}@binary_new'] = binary_new
            count_new += 1
        if binary_old != 'None':
            allFunctions[f'{diff_fp}@binary_old'] = binary_old
            count_old += 1
        total_functions += 1

    print(f'new: {count_new}')
    print(f'old: {count_old}')
    print('total_functions:', total_functions)
    diff_output = []
    binary_output = []
    attention_mask = []
    psu_attention_mask = []
    functionNames = list(allFunctions.keys())
    total = 0
    count = batch_size
    for key in tqdm(output_d):
        # while True:
        #     unrelated_example = r.choice(functionNames)
        #     fname_old = key + '@binary_old'
        #     fname_new = key + '@binary_new'
        #     if unrelated_example != fname_old and unrelated_example != fname_new:
        #         break
        funcList = output_d[key]

        for i in range(0,len(funcList)):
            token_id_list = copy.deepcopy(funcList[i])
            attention_mask.append([1] * len(token_id_list))
            if (l := len(token_id_list)) < func_len:
                token_id_list += [s_pad_id] * (func_len - l)
                attention_mask[-1] += [0] * (func_len - l)
            #print(len(token_id_list))
            diff_output.append(token_id_list)

        count -= 1
        if count == 0 or total == len(output_d) - 1:
            total += 1
            out_data_diff = np.asarray(diff_output, dtype=np.int64)
            out_attention_mask = np.asarray(attention_mask,dtype=np.int64)

            diff_output = []
            attention_mask = []
            count = batch_size

            yield out_data_diff, out_attention_mask

def test_data_loader_codeBert_recall(tokenizer, func_len, batch_size, mode, poolsize):
    pad_id = tokenizer.pad_token_id
    all_functions = {}
    positive_pairs = {}
    diff_functions = []
    total_functions = 0

    for path, token_id_list, version in yield_functions(shuffle=True, seed=seed, split_ratio=0.8, mode=mode,testing=True):
        func_name = f'{path}@{version}'
        all_functions[func_name] = token_id_list
        if version == 'diff' or version == 'new':
            positive_pairs[func_name] = token_id_list
        else:
            #positive_pairs[func_name] = token_id_list
            diff_functions.append(positive_pairs)
            positive_pairs = {}
        total_functions += 1
        #print(f'{k[-4]}: add function to {func_name} Length:{len(output_d[k[-4]][func_name])}')
    padding_list = []
    output = []
    labels = []

    processed_functions = 0
    #count = 0
    for positive_pair in tqdm(diff_functions):
        pk = list(positive_pair.keys())
        function_names = list(all_functions.keys())
        start = True
        while start:
            func_list = [positive_pair[pk[0]],positive_pair[pk[1]]]
            while len(func_list) < batch_size:
                fname = random.choice(function_names)
                if fname not in positive_pair:
                    func_list.append(all_functions[fname])
                    function_names.remove(fname)

            for token_id_list in func_list:
                copy_token_id_list = copy.deepcopy(token_id_list)
                padding_list.append([True] * len(copy_token_id_list))

                if (l := len(copy_token_id_list)) < func_len:
                    copy_token_id_list += [pad_id] * (func_len - l)
                    padding_list[-1] += [False] * (func_len - l)
                output.append(copy_token_id_list)

            processed_functions += len(output) - 2
            data = np.asarray(output, dtype=np.int64)
            attention_mask = np.asarray(padding_list, dtype=bool)
            padding_list.clear()
            output.clear()
            if processed_functions % poolsize == 0:
                start = False
            yield data, attention_mask, labels, pk[0]

In [ ]:
class WarmupLinearSchedule(LambdaLR):
    """ Linear warmup and then linear decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Linearly decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps.
    """
    def __init__(self, optimizer, warmup_steps, t_total, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        super(WarmupLinearSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        return max(0.0, float(self.t_total - step) / float(max(1.0, self.t_total - self.warmup_steps)))

class Model(nn.Module):
    def __init__(self, encoder, config):
        super(Model, self).__init__()
        #self.encoder = encoder.get_encoder()
        self.encoder = encoder
        self.config = config
        self.func_emb1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.func_emb2 = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)

    def forward(self, x, attention_mask):
        x = self.encoder(x,attention_mask=attention_mask)
        # (batch_size, seq_len, hidden_size)
        #x = x.last_hidden_state[:,0,:]
        #x = self.func_emb1(self.dropout(x))
        #x = F.gelu(x)
        #x = self.func_emb2(self.dropout(x))
        # avg pooler: convert (batch_size, seq_len, hidden_size) -> (batch_size,hidden_size)
        x = x.last_hidden_state
        x = (x * attention_mask.unsqueeze(-1)).sum(1) /  attention_mask.sum(-1).unsqueeze(-1)
        return x

def cosine_similarity_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    # clamp(min=eps) assign eps if encounter zeros
    return torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

def non_diagonal(x):
    sz = x.size()
    assert sz[0] == sz[1]
    n = sz[0]
    return x.view(-1)[1:].view(n - 1, n + 1)[:, :-1].reshape(n, n - 1)

In [ ]:
def finetune():
    torch.cuda.empty_cache()
    model = Model(pretrain_model,config).to(device)
    lr = 5e-5
    total_epoch = 5
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    totalInstance = len(getAllPath())
    batch_size = 8
    total_train = (totalInstance * 0.8 ) / batch_size
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=total_epoch * total_train * 0.1, t_total=total_epoch * total_train)
    cur_step = 0
    global_loss = 0.0

    best = 0
    for epoch in range(total_epoch):
        for data, attention_mask in finetune_data_loader_codeBert(tokenizer, batch_size, 512, 'train'):
            data = torch.as_tensor(data, device=device)
            attention_mask = torch.as_tensor(attention_mask, device=device)

            output = model(data,attention_mask)
            row = torch.arange(0,output.size(0),3)
            col = torch.arange(0,output.size(0))
            col = col[col % 3 != 0] # select all cols != diff

            sim = cosine_similarity_torch(output)

            sim = sim[row,:]
            sim = sim[:,col]
            sim = sim / 0.05

            labels = torch.arange(0, len(col), 2, device=device)
            loss = F.cross_entropy(sim, labels)
            #loss = torch.mean(loss) / accum_iter
            loss.backward()

            #if (cur_step + 1) % accum_iter == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            cur_step += 1
            if cur_step % 500 == 0:
                print(f'{epoch=}, {cur_step=}, loss={loss.item()}')
                if os.path.exists(model_path):
                  torch.save(model.state_dict(), f'model_state/codeBert_finetune.dat')
            global_loss += loss.item()
        global_loss /= totalInstance
        print(f'{epoch=}, {cur_step=}, global_loss={global_loss}')
        recall = test(model,epoch,global_loss)
        model.train()

        # if best < recall:
        #     best = recall

def test(model=None,epoch=None,loss=None):
    model.eval()
    recall_1 = 0
    total = 0
    poolsize = 2000
    step = 0
    count = 0

    with torch.no_grad():
        for data, attention_mask, labels, diff_name in test_data_loader_codeBert_recall(tokenizer, 512, 52, 'test',poolsize):
            data = torch.as_tensor(data, device=device)
            attention_mask = torch.as_tensor(attention_mask, device=device)
            output = model(data, attention_mask)
            sim = F.cosine_similarity(output[0].unsqueeze(0),output[1:])
            indx = torch.argmax(sim)

            if indx == 0:
                count += sim.size(0) - 1

            step += 1

            if step * 50 == poolsize:
                if count == poolsize:
                    recall_1 += 1
                #print(f'Diff function: {diff_name}, process {total + 1} diff functions, step: {step}, count: {count}/{poolsize}')
                total += 1
                step = 0
                count = 0


    #print(f'epoch: {epoch}, average recall@1 with poolsize {poolsize}: {(recall_1/total):.4f}, loss:{loss}')
    print(f'average recall@1 with poolsize {poolsize}: {(recall_1/total):.4f}')

    return recall_1 / total

In [ ]:
finetune()

13269it [00:00, 17955.78it/s]


14642


13269it [00:00, 18209.58it/s]


14642


100%|██████████| 11713/11713 [01:07<00:00, 174.34it/s]


...

epoch=0, cur_step=791, global_loss=0.04298840349586165


13269it [00:01, 8748.53it/s]


14642


100%|██████████| 2929/2929 [00:52<00:00, 56.31it/s]


KeyboardInterrupt: ignored

In [ ]:
test()

6885it [00:00, 29010.97it/s]
100%|██████████| 767/767 [00:03<00:00, 203.09it/s]


average recall@1 with poolsize 500: 0.9712


0.9712041884816754